In [1]:
!pip install numpy == 1.26.0
!pip install torch == 2.2.2 torchvision torchaudio ---url https://download.pytorch.org/whl/cpu
!pip install torchtext
!pip install torchdata
!pip install portalocker == 2.8.2
!pip install pandas ==2.2.1
!pip install matplotlib ==3.9.0 scikit-learn ==1.5.0
!pip install transformers==4.35.2

zsh:1: = not found
zsh:1: = not found
zsh:1: = not found
zsh:1: =2.2.1 not found
zsh:1: =3.9.0 not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


## Importing Libraries

In [2]:
import torchtext
from torchtext.datasets import multi30k,Multi30k
import torch
from typing import Iterable, List
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.nn import Transformer
import math
from torchtext.vocab import Vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torchtext.datasets import IMDB,PennTreebank
import time



import warnings
def warn(*args,**kwargs):
    pass
warnings.warn = warn
warnings.filterwarnings('ignore')


In [3]:
train_iter, valid_iter = IMDB()

In [4]:
data_iter = iter(train_iter)
next(data_iter)
next(data_iter)
next(data_iter)

(1,
 "If only to avoid making this type of film in the future. This film is interesting as an experiment but tells no cogent story.<br /><br />One might feel virtuous for sitting thru it because it touches on so many IMPORTANT issues but it does so without any discernable motive. The viewer comes away with no new perspectives (unless one comes up with one while one's mind wanders, as it will invariably do during this pointless film).<br /><br />One might better spend one's time staring out a window at a tree growing.<br /><br />")

### Data Processing

* Special Symbols and Indices
    * UNK_IDX
    * PAD_IDX
    * EOS_IDX

* yield tokens
* create vocabs
* Default index for unknown tokens
* Text to index
* Index to en


In [5]:
UNK_IDX, PAD_IDX, EOS_IDX = 0,1,2
special_symbols = ['<unk>', '<pad>', '<|endoftext|>']

In [6]:
tokenizer = get_tokenizer('basic_english')

In [7]:
def yield_tokens(data_iter):
    for _,data_sample in data_iter:
        yield tokenizer(data_sample)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials = special_symbols, special_first=True)
vocab.set_default_index(UNK_IDX)

In [10]:
text_to_index = lambda text: [vocab[token] for token in tokenizer(text)]
index_to_english = lambda seq_en:" ".join([vocab.get_itos()[index] for index in seq_en])

In [11]:
index_to_english(torch.tensor([0,1,2]))

'<unk> <pad> <|endoftext|>'

In [14]:
items = list(vocab.get_itos())[:30]
items

['<unk>',
 '<pad>',
 '<|endoftext|>',
 '.',
 'the',
 ',',
 'a',
 'and',
 "'",
 'of',
 'to',
 'is',
 'it',
 'i',
 'in',
 'this',
 'that',
 's',
 'was',
 'movie',
 'for',
 'but',
 'with',
 'as',
 't',
 'film',
 'you',
 ')',
 'on',
 '(']

### Collate Function

In [17]:
def get_sample(block_size, text):
    # block size: Context size, how many tokens/indices the model will look after at a time
    # text: text is the long sequence of indices.
    # Determine the length of the input text
    sample_leg = len(text) # Get the total number of tokens in the text

    # Calculate the stopping point for randomly selecting a sample
    # This ensures the selected sample doesn't exceed the text
    random_sample_stop = sample_leg - block_size
    # You're calculating how far you can go into the text while still being able to extract a full block_size chunk.


    if random_sample_stop >= 1:
        random_start = torch.randint(0,random_sample_stop,size = (1,)).item()

        stop = random_start + block_size

        # Create the input and the target sequence
        src_seq = text[random_start:stop]

        trg_seq = text[random_start+1:stop+1]


    # Handle the case where the text length is exactly equal to the block size or less than that

    elif random_sample_stop<=0:

        random_start = 0
        stop = sample_leg

        src_seq = text[random_start:stop]
        trg_seq = text[random_start+1:stop]

        # Append an empty string to maintain the sequence alignmenbt
        trg_seq.append('<|endoftext>|')


    return src_seq,trg_seq    

In [18]:
BATCH_SIZE = 1
batch_of_tokens = []


for i in range(BATCH_SIZE):
    _,text = next(iter(train_iter))
    batch_of_tokens.append(tokenizer(text))

In [21]:
batch_of_tokens

[['i',
  'rented',
  'i',
  'am',
  'curious-yellow',
  'from',
  'my',
  'video',
  'store',
  'because',
  'of',
  'all',
  'the',
  'controversy',
  'that',
  'surrounded',
  'it',
  'when',
  'it',
  'was',
  'first',
  'released',
  'in',
  '1967',
  '.',
  'i',
  'also',
  'heard',
  'that',
  'at',
  'first',
  'it',
  'was',
  'seized',
  'by',
  'u',
  '.',
  's',
  '.',
  'customs',
  'if',
  'it',
  'ever',
  'tried',
  'to',
  'enter',
  'this',
  'country',
  ',',
  'therefore',
  'being',
  'a',
  'fan',
  'of',
  'films',
  'considered',
  'controversial',
  'i',
  'really',
  'had',
  'to',
  'see',
  'this',
  'for',
  'myself',
  '.',
  'the',
  'plot',
  'is',
  'centered',
  'around',
  'a',
  'young',
  'swedish',
  'drama',
  'student',
  'named',
  'lena',
  'who',
  'wants',
  'to',
  'learn',
  'everything',
  'she',
  'can',
  'about',
  'life',
  '.',
  'in',
  'particular',
  'she',
  'wants',
  'to',
  'focus',
  'her',
  'attentions',
  'to',
  'making',
 